In [1]:
import torch
from pathlib import Path
from monai.utils        import set_determinism  
from split_data         import split_data
from transforms         import get_transforms
from model              import ResidualAttention3DUnet, MTLResidualAttention3DUnet, MTLResidualAttentionRecon3DUnet
from train_model        import train_model
from test_model         import test_model
from train_model_base   import train_model_base
from test_model_base    import test_model_base

In [2]:
# Choose whether to train and/or test model(s)
TRAIN           = 1
TEST            = 1

# Choose which models to test
BASE_CASE       = 1
AUX_SEGMENT     = 1
AUX_RECONSTRUCT = 1

# Parameters
params = {
    'BATCH_SIZE':       2,
    'MAX_EPOCHS':       2,
    'VAL_INTERVAL':     1,
    'PRINT_INTERVAL':   1
}

# Set deterministic training for reproducibility
set_determinism(seed = 2056)

# Path to data
img_path = Path("../data")
train_files, val_files, test_files = split_data(img_path, scale=28)

# Create transforms for training
train_transforms, val_transforms, pred_main, label_main, pred_aux, label_aux = get_transforms()

# Use CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define organ names in the segmentation task
all_organs =  ["Background", "Bladder", "Bone", "Obturator internus", "Transition zone", "Central gland", "Rectum", "Seminal vesicle", "Neurovascular bundle"]
organs = {
    'all': all_organs,
    'main': ["Transition zone", "Central gland"],
    'aux': [],
    'dict': {organ: idx for idx, organ in enumerate(all_organs)}
    }

----------------------------------------
Splitting data into train-validate-test sets...
The file does not exist
The file does not exist
Images have been divided into train-validate-test sets.
Total number of images:  585
Number of images train-validate-test:  16 - 2 - 2
----------------------------------------
Creating transformations...
Transforms have been defined.


## BASE CASE

In [ ]:
model  = ResidualAttention3DUnet(in_channels = 1, out_channels = len(organs['main'])+1, device=device).to(device) 

if TRAIN:
    torch.cuda.empty_cache()
    train_model_base(model, device, params, train_files, train_transforms, val_files, val_transforms, organs, pred_main, label_main)

In [ ]:
if TEST:        
        torch.cuda.empty_cache()
        test_model_base(model, device, params, test_files, val_transforms, organs, pred_main, label_main)

## AUXILIARY - SEGMENT

In [7]:
organs['aux']  = ["Rectum", "Seminal vesicle", "Neurovascular bundle"]
params['TASK'] = 'SEGMENT'
model = MTLResidualAttention3DUnet(in_channels = 1, main_out_channels = len(organs['main'])+1, aux_out_channels = len(organs['aux'])+1, device=device).to(device) 

if TRAIN:
    torch.cuda.empty_cache()
    train_model(model, device, params, train_files, train_transforms, val_files, val_transforms, organs, pred_main, label_main, pred_aux, label_aux)

--------------------
Starting model training...
--------------------
Epoch 1 / 2
2023-04-25 17:08:54,091 - pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
/opt/anaconda3/lib/python3.9/site-packages/torch/_tensor.py:1295: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/opt/anaconda3/lib/python3.9/site-packages/monai/data/__init__.py:127: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if storage.is_cuda:



Epoch 1 average loss for main task: 1.0000

Epoch 1 average loss for aux task: 1.0000

Epoch 1 average total loss for both tasks: 2.6000
----------------------------------------
Testing on validation data...


/opt/anaconda3/lib/python3.9/site-packages/torch/_tensor.py:1295: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/opt/anaconda3/lib/python3.9/site-packages/monai/data/__init__.py:127: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if storage.is_cuda:


saved new best metric model

Current epoch: 1 current mean dice for main task: 0.0000
Best mean dice for main task: 0.0000 at epoch: 1
Current epoch: 1 current mean metric for aux task: 0.0000
Done training! Best mean dice: 0.0000 at epoch: 1


In [8]:
if TEST:
    torch.cuda.empty_cache()
    test_model(model, device, params, test_files, val_transforms, organs, pred_main, label_main, pred_aux, label_aux)

----------------------------------------
Starting model testing...


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


2023-04-25 17:16:41,198 - pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


/opt/anaconda3/lib/python3.9/site-packages/torch/_tensor.py:1295: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/opt/anaconda3/lib/python3.9/site-packages/monai/data/__init__.py:127: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if storage.is_cuda:



Mean dice for main task: 0.0000
Mean metric for aux task: 0.0000


## AUXILIARY - RECONSTRUCT

In [4]:
organs['aux'] = []
params['TASK'] = 'RECONSTRUCT'
    
model = MTLResidualAttentionRecon3DUnet(in_channels = 1, out_channels = len(organs['main'])+1, device=device).to(device) 

if TRAIN:
    torch.cuda.empty_cache()
    train_model(model, device, params, train_files, train_transforms, val_files, val_transforms, organs, pred_main, label_main, pred_aux, label_aux)

--------------------
Starting model training...
--------------------
Epoch 1 / 2


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


2023-04-25 20:04:50,226 - pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


/opt/anaconda3/lib/python3.9/site-packages/torch/_tensor.py:1295: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/opt/anaconda3/lib/python3.9/site-packages/monai/data/__init__.py:127: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if storage.is_cuda:
/opt/anaconda3/lib/python3.9/site-packages/monai/data/__init__.py:120: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are usi


Epoch 1 average loss for main task: 1.0000

Epoch 1 average loss for aux task: 0.1795

Epoch 1 average total loss for both tasks: 1.3693
----------------------------------------
Testing on validation data...


/opt/anaconda3/lib/python3.9/site-packages/torch/_tensor.py:1295: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/opt/anaconda3/lib/python3.9/site-packages/monai/data/__init__.py:127: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if storage.is_cuda:


In [ ]:
organs['aux'] = []
params['TASK'] = 'RECONSTRUCT'
model = MTLResidualAttentionRecon3DUnet(in_channels = 1, out_channels = len(organs['main'])+1, device=device).to(device) 

if TEST:
    torch.cuda.empty_cache()
    test_model(model, device, params, test_files, val_transforms, organs, pred_main, label_main, pred_aux, label_aux)

RuntimeError: Error(s) in loading state_dict for MTLResidualAttentionRecon3DUnet:
	Missing key(s) in state_dict: "attention_blocks.0.W_g.up_sample.weight", "attention_blocks.0.W_g_norm.weight", "attention_blocks.0.W_g_norm.bias", "attention_blocks.0.W_x.weight", "attention_blocks.0.W_x_norm.weight", "attention_blocks.0.W_x_norm.bias", "attention_blocks.0.phi.weight", "attention_blocks.0.final_norm.weight", "attention_blocks.0.final_norm.bias", "attention_blocks.1.W_g.up_sample.weight", "attention_blocks.1.W_g_norm.weight", "attention_blocks.1.W_g_norm.bias", "attention_blocks.1.W_x.weight", "attention_blocks.1.W_x_norm.weight", "attention_blocks.1.W_x_norm.bias", "attention_blocks.1.phi.weight", "attention_blocks.1.final_norm.weight", "attention_blocks.1.final_norm.bias", "attention_blocks.2.W_g.up_sample.weight", "attention_blocks.2.W_g_norm.weight", "attention_blocks.2.W_g_norm.bias", "attention_blocks.2.W_x.weight", "attention_blocks.2.W_x_norm.weight", "attention_blocks.2.W_x_norm.